In [ ]:
# colab에서 실행되었습니다

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import datetime
from sklearn.model_selection import train_test_split

import torch
import re
import os
import shutil
import glob
import zipfile
import cv2

import tensorflow as tf
from tensorflow import keras

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('device :', device)

device : cpu


# 1. 구글 드라이브 마운트

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 작업 경로 변경

# 현재 작업 디렉토리 확인
print('현재 작업 경로 :', os.getcwd())

# 작업 디렉토리 변경
os.chdir('/content/drive/MyDrive/BoneAge')
print('변경된 작업 경로 :', os.getcwd())

현재 작업 경로 : /content
변경된 작업 경로 : /content/drive/MyDrive/BoneAge


# 2. 학습 데이터 생성

In [ ]:
# crop 이미지 경로
LMCP_path = '/content/drive/MyDrive/BoneAge/yolov5/runs/detect/exp4/crops/LMCP/'

MMCP_path = '/content/drive/MyDrive/BoneAge/yolov5/runs/detect/exp4/crops/MMCP/'

TMCP_path = '/content/drive/MyDrive/BoneAge/yolov5/runs/detect/exp4/crops/TMCP/'

CARPAL_path = '/content/drive/MyDrive/BoneAge/yolov5/runs/detect/exp4/crops/CARPAL/'

df = pd. read_csv('./Data/df.csv')
df.head()


,Unnamed: 0,No.,Group,등록번호,생년월일,성별,진료의,검사 시 나이,신장,체중,BMI,처방일자,시행일자,BA 1,BA 2,BA
0,0,332_M,76,4173152,2002-04-17,1,2,9.950685,136.4,42.4,22.8,2012-03-27,2012-03-27,8.75,9.50,9.125
1,1,333_M,1439,7968480,2007-02-27,1,2,9.956164,148.9,44.3,20.0,2017-02-08,2017-02-08,12.50,12.25,12.375
2,2,334_M,1709,8269414,2008-03-07,1,2,9.964384,130.4,30.6,18.0,2017-08-16,2018-02-20,9.25,8.25,8.750
3,3,335_M,1710,8269417,2008-03-07,1,2,9.964384,128.1,26.2,16.0,2017-08-16,2018-02-20,9.25,8.25,8.750
4,4,336_M,364,5952933,2003-02-12,1,1,9.972603,128.7,32.7,19.8,2012-05-10,2013-01-30 00:00:00,11.00,11.25,11.125


In [ ]:
def optimize_img(r1,r2,r3,r4):
    tmp_binary_img = []
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)) #CLAHE 생성
    for img in [r1,r2,r3,r4]:
        if img is not None:
            # resized_img = cv2.resize(img,(1000,400)) # (400, 500)
            blured_img = cv2.GaussianBlur(img,(5,5),0)
            clahed_img = clahe.apply(blured_img)          #CLAHE 적용
            # _,binary_img = cv2.threshold(clahed_img,clahed_img.mean()*1.25,255,cv2.THRESH_BINARY)

            target_length = 500
            (original_height, original_width) = clahed_img.shape
            # 가로세로 비율을 유지하면서 긴 부분을 target_length로 조정합니다.
            if original_width > original_height:
                # 가로가 길 경우
                new_width = target_length
                new_height = int((new_width / original_width) * original_height)
            else:
                # 세로가 길 경우
                new_height = target_length
                new_width = int((new_height / original_height) * original_width)

            # 이미지 크기 조정
            resized_image = cv2.resize(clahed_img, (new_width, new_height))

            # 최종 이미지 크기
            target_size = 600
            old_size = resized_image.shape

            # 새로운 이미지 생성 (검은색 배경)
            new_image = np.zeros((target_size, target_size), dtype=np.uint8)

            # 이미지 중앙에 배치하기 위한 좌표 계산
            start_x = (target_size - old_size[1]) // 2
            start_y = (target_size - old_size[0]) // 2

            # 원본 이미지를 중앙에 배치
            new_image[start_y:start_y+old_size[0], start_x:start_x+old_size[1]] = resized_image

            tmp_binary_img.append(new_image)

    return np.array(tmp_binary_img)

In [ ]:
X_data, y_data = [], []

for i in range(len(df)):
    filename = df["No."][i] + ".jpg"
    LMCP_img = cv2.imread(os.path.join(LMCP_path, filename), cv2.IMREAD_GRAYSCALE)
    MMCP_img = cv2.imread(os.path.join(MMCP_path, filename), cv2.IMREAD_GRAYSCALE)
    TMCP_img = cv2.imread(os.path.join(TMCP_path, filename), cv2.IMREAD_GRAYSCALE)
    CARPAL_img = cv2.imread(os.path.join(CARPAL_path, filename), cv2.IMREAD_GRAYSCALE)

    # 모든 이미지를 동일한 크기로 조정합니다.
    LMCP_img = cv2.resize(LMCP_img, (400, 400))
    MMCP_img = cv2.resize(MMCP_img, (400, 400))
    TMCP_img = cv2.resize(TMCP_img, (400, 400))
    CARPAL_img = cv2.resize(CARPAL_img, (400, 400))

    optimized_img = optimize_img(LMCP_img, MMCP_img, TMCP_img, CARPAL_img)

    row1 = np.vstack((optimized_img[0], optimized_img[1]))
    row2 = np.vstack((optimized_img[2], optimized_img[3]))

    combined_image = np.hstack((row1, row2))
    combined_image = cv2.resize(combined_image, (256, 256))

    X_data.append(combined_image)
    y_data.append(df["BA"][i])

X_data, y_data = np.array(X_data), np.array(y_data)

# 3. TJ-Net 모델 생성

In [ ]:
# separable 합성곱 함수
def separable_conv(x, inchannel,outchannel):
  x = keras.layers.Conv2D(inchannel, (3,3), strides=1, padding="same")(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Conv2D(outchannel, (1,1), strides=1, padding="same")(x)
  x = keras.layers.BatchNormalization()(x)
  return x

In [ ]:
# model middle_flow 함수
def middle_flow(input_x):
  # encoder
  x = keras.layers.MaxPool2D((2,2),padding="same")(input_x)
  x = resiual_units(x)
  x = keras.layers.MaxPool2D((2,2),padding="same")(x)
  x = resiual_units(x)
  x = keras.layers.MaxPool2D((2,2),padding="same")(x)
  x = resiual_units(x)

  # decoder
  x = resiual_units(x)
  x = keras.layers.UpSampling2D((2,2),interpolation='bilinear')(x)
  x = resiual_units(x)
  x = keras.layers.UpSampling2D((2,2),interpolation='bilinear')(x)
  x = resiual_units(x)
  x = keras.layers.UpSampling2D((2,2),interpolation='bilinear')(x)

  x = separable_conv(x,x.shape[-1],512)
  x = separable_conv(x,x.shape[-1],512)

  # sigmoid
  x = keras.activations.sigmoid(x)
  x = keras.layers.Multiply()([input_x,x])
  x = keras.layers.Add()([input_x,x])

  x = resiual_units(x)

  return x

In [ ]:
# resiual_units 함수
def resiual_units(input_x):
  x = keras.layers.ReLU()(input_x)
  x = separable_conv(x,x.shape[-1],128)
  x = keras.layers.BatchNormalization()(x)

  x = keras.layers.ReLU()(x)
  x = separable_conv(x,x.shape[-1],256)
  x = keras.layers.BatchNormalization()(x)

  x = keras.layers.ReLU()(x)
  x = separable_conv(x,x.shape[-1],512)
  x = keras.layers.BatchNormalization()(x)

  input_x = keras.layers.Add()([x,input_x])

  return input_x

In [ ]:
# 골연령 측정 모델
# entry flow model
input = keras.Input(shape=(256,256,1))
x = keras.layers.Conv2D(32, (3,3), strides = 2)(input)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)

x = keras.layers.Conv2D(64, (3,3), strides=1)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)

# 첫번째
x1 = keras.layers.Conv2D(128,(1,1),strides=2)(x)
x1 = keras.layers.BatchNormalization()(x1)

x = separable_conv(x,x.shape[-1],128)
x = keras.layers.ReLU()(x)

x = separable_conv(x,x.shape[-1],128)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPool2D((2,2),2,padding="same")(x)

x = keras.layers.Add()([x,x1])

# 2번째
x1 = keras.layers.Conv2D(512,(1,1),strides=2)(x)
x1 = keras.layers.BatchNormalization()(x1)

x = separable_conv(x,x.shape[-1],512)
x = keras.layers.ReLU()(x)

x = separable_conv(x,x.shape[-1],512)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPool2D((2,2),2,padding="same")(x)

x = keras.layers.Add()([x,x1])


# middle flow model
x = middle_flow(x)


# exit flow model
x1 = keras.layers.Conv2D(1024,(1,1),strides=2)(x)

x = keras.layers.ReLU()(x)
x = separable_conv(x,x.shape[-1],728)
x = keras.layers.ReLU()(x)
x = separable_conv(x,x.shape[-1],1024)
x = keras.layers.MaxPool2D((2,2),2,padding="same")(x)

x = keras.layers.Add()([x,x1])


x = separable_conv(x,x.shape[-1],1536)
x = keras.layers.ReLU()(x)
x = separable_conv(x,x.shape[-1],2048)
x = keras.layers.ReLU()(x)

x = keras.layers.GlobalAvgPool2D()(x)

x = keras.layers.Dense(1000,activation='relu')(x)
x = keras.layers.Dense(256,activation='relu')(x)
x = keras.layers.Dense(1)(x)

model = keras.models.Model(input,x)

# CNN 모델 구조 확정 및 컴파일 진행
model.compile(optimizer='adam', loss='mae', metrics=['mae','mse'], run_eagerly=True)

In [ ]:
model.save('./tjnet_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 127, 127, 32)         320       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 127, 127, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 127, 127, 32)         0         ['batch_normalization[0][0

# 4. Train, Test, Validation 데이터셋 나누기

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data.reshape(1235, 256, 256, 1),
                                                    y_data, random_state=42, test_size=0.2)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42, test_size = 0.2)

In [ ]:
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

filename = 'checkpoint-150-epochs-8-batchs.h5'
checkpoint = ModelCheckpoint(filename,              # file명을 지정합니다
                             mointor='val_loss',    # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,             # 로그를 출력합니다
                             save_best_only = True, # 가장 best 값만 저장합니다
                             mode='auto')           # auto는 알아서 best를 찾습니다. min/max


In [ ]:
model.fit(X_train, y_train, batch_size=8, epochs=150, callbacks=checkpoint, validation_data=(X_val, y_val))

Epoch 1/150


99/99 [==============================] - ETA: 0s - loss: 2.4596 - mae: 2.4596 - mse: 9.9500
Epoch 1: val_loss improved from inf to 1063.02710, saving model to checkpoint-150-epochs-8-batchs.h5
99/99 [==============================] - 147s 903ms/step - loss: 2.4596 - mae: 2.4596 - mse: 9.9500 - val_loss: 1063.0271 - val_mae: 1063.0271 - val_mse: 1137883.7500
Epoch 2/150
99/99 [==============================] - ETA: 0s - loss: 1.8541 - mae: 1.8541 - mse: 5.7899
Epoch 2: val_loss improved from 1063.02710 to 152.20944, saving model to checkpoint-150-epochs-8-batchs.h5
99/99 [==============================] - 93s 938ms/step - loss: 1.8541 - mae: 1.8541 - mse: 5.7899 - val_loss: 152.2094 - val_mae: 152.2094 - val_mse: 24087.1523
Epoch 3/150
99/99 [==============================] - ETA: 0s - loss: 1.8088 - mae: 1.8088 - mse: 5.8307
Epoch 3: val_loss improved from 152.20944 to 33.13047, saving model to checkpoint-150-epochs-8-batchs.h5
99/99 [==============================] - 88s 887ms/step - 

In [ ]:
from keras.models import load_model
model = load_model('./checkpoint-150-epochs-8-batchs.h5')

In [ ]:
model.fit(X_train, y_train, batch_size=8, epochs=150, callbacks=checkpoint, validation_data=(X_val, y_val))

Epoch 1/150
99/99 [==============================] - ETA: 0s - loss: 0.4393 - mae: 0.4393 - mse: 0.3176
Epoch 1: val_loss improved from inf to 1.78005, saving model to checkpoint-150-epochs-8-batchs.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


99/99 [==============================] - 146s 807ms/step - loss: 0.4393 - mae: 0.4393 - mse: 0.3176 - val_loss: 1.7800 - val_mae: 1.7800 - val_mse: 4.3619
Epoch 2/150
99/99 [==============================] - ETA: 0s - loss: 0.3714 - mae: 0.3714 - mse: 0.2404
Epoch 2: val_loss improved from 1.78005 to 0.74551, saving model to checkpoint-150-epochs-8-batchs.h5
99/99 [==============================] - 64s 652ms/step - loss: 0.3714 - mae: 0.3714 - mse: 0.2404 - val_loss: 0.7455 - val_mae: 0.7455 - val_mse: 0.8840
Epoch 3/150
99/99 [==============================] - ETA: 0s - loss: 0.3550 - mae: 0.3550 - mse: 0.2079
Epoch 3: val_loss did not improve from 0.74551
99/99 [==============================] - 49s 497ms/step - loss: 0.3550 - mae: 0.3550 - mse: 0.2079 - val_loss: 1.2404 - val_mae: 1.2404 - val_mse: 2.2930
Epoch 4/150
99/99 [==============================] - ETA: 0s - loss: 0.3544 - mae: 0.3544 - mse: 0.2013
Epoch 4: val_loss did not improve from 0.74551
99/99 [======================

In [ ]:
test_prediction = model.predict(X_test)

8/8 [==============================] - 381s 47s/step


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print('mae : {:.3f}'.format(mean_absolute_error(test_prediction, y_test)))
print('rmse : {:.3f}'.format(np.sqrt(mean_squared_error(test_prediction, y_test))))

mae : 0.589
rmse : 0.747


# 5. Result

* batchsize : 8
* epoch : 150
* mae : 0.589, rmse : 0.747
* 즉, 전문의 판단과 예측 모델의 판단 결과의 차이는 약 0.589년 차이를 보입니다.
* 기존 데이터의 BA 단위는 year 이므로, 단위를 months로 변경
* 약 7.1개월 차이를 보입니다.

